In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import vectorbt as vbt
from datetime import datetime, timedelta
import time
import requests
import os
import time
import concurrent.futures
import warnings
warnings.filterwarnings("ignore")


In [2]:
# CoinMarketCap API setup
CMC_API_KEY = 'f13d0cc7-704c-407d-81ab-f0118c933b08'
CMC_URL = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
CMC_HEADERS = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': CMC_API_KEY,
}
limit=300
params = {
        'start': 1,
        'limit': limit,
        'sort': 'market_cap',
        'sort_dir': 'desc',
        'cryptocurrency_type': 'all',
    }
response = requests.get(CMC_URL, headers=CMC_HEADERS, params=params)
print(response)
data = response.json()['data']
print(data)
df = pd.json_normalize(data, sep='_')
df['quote_USD_fully_diluted_market_cap'] = pd.to_numeric(df['quote_USD_fully_diluted_market_cap'], errors='coerce')
df['Symbol Name']=df['symbol']+'USDT'
selected=df['Symbol Name'].unique()


<Response [200]>
[{'id': 1, 'name': 'Bitcoin', 'symbol': 'BTC', 'slug': 'bitcoin', 'num_market_pairs': 11071, 'date_added': '2010-07-13T00:00:00.000Z', 'tags': ['mineable', 'pow', 'sha-256', 'store-of-value', 'state-channel', 'coinbase-ventures-portfolio', 'three-arrows-capital-portfolio', 'polychain-capital-portfolio', 'binance-labs-portfolio', 'blockchain-capital-portfolio', 'boostvc-portfolio', 'cms-holdings-portfolio', 'dcg-portfolio', 'dragonfly-capital-portfolio', 'electric-capital-portfolio', 'fabric-ventures-portfolio', 'framework-ventures-portfolio', 'galaxy-digital-portfolio', 'huobi-capital-portfolio', 'alameda-research-portfolio', 'a16z-portfolio', '1confirmation-portfolio', 'winklevoss-capital-portfolio', 'usv-portfolio', 'placeholder-ventures-portfolio', 'pantera-capital-portfolio', 'multicoin-capital-portfolio', 'paradigm-portfolio', 'bitcoin-ecosystem', 'ftx-bankruptcy-estate'], 'max_supply': 21000000, 'circulating_supply': 19704265, 'total_supply': 19704265, 'infinite_

In [3]:
def Pull_Historical_data(symbols, interval="1m"):
    results = {}
    def fetch_klines(symbol, interval, start_timestamp, end_timestamp, limit=1000):
        url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&startTime={start_timestamp}&endTime={end_timestamp}&limit={limit}"
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            # print(f"Failed to retrieve data from Binance API: {response.status_code}")
            return []

    def pull_historical_data(symbol, interval="1m"):
        start_time = "2023-01-27 19:10:00"
        end_time = "2024-05-26 17:12:00"

        start_timestamp = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S"))) * 1000
        end_timestamp = int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S"))) * 1000

        limit = 1000
        klines = []

        current_timestamp = start_timestamp

        with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
            future_to_url = {
                executor.submit(fetch_klines, symbol, interval, current_timestamp, min(current_timestamp + limit * 60 * 1000, end_timestamp), limit): current_timestamp
                for current_timestamp in range(start_timestamp, end_timestamp, limit * 60 * 1000)
            }
            for future in concurrent.futures.as_completed(future_to_url):
                data = future.result()
                for kline in data:
                    klines.append([
                        kline[0],  # Open time
                        # kline[2],  # High
                        # kline[3],  # Low
                        kline[4],  # Close
                        # kline[5],  # Volume
                        # kline[7],  # Number of trades
                        # kline[8],  # Taker buy base asset volume
                    ])

        df = pd.DataFrame(klines, columns=[
            "Open time",
            # "High",
            # "Low",
            "Close"
            #,
            # "Volume",
            # "Number of trades",
            # "Taker buy base asset volume",
        ])

        df["Open time"] = pd.to_datetime(df["Open time"], unit='ms')
        df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
        # df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')

        return df

    for symbol in symbols:
        print(f"Pulling data for {symbol}...")
        df = pull_historical_data(symbol, interval)
        results[symbol] = df
        print(results.keys())

    return results

data = Pull_Historical_data(selected, '1d')
for symbol, df in data.items():
    print(f"Data for {symbol}:")
    print(df)

Pulling data for BTCUSDT...
dict_keys(['BTCUSDT'])
Pulling data for ETHUSDT...
dict_keys(['BTCUSDT', 'ETHUSDT'])
Pulling data for USDTUSDT...
dict_keys(['BTCUSDT', 'ETHUSDT', 'USDTUSDT'])
Pulling data for BNBUSDT...
dict_keys(['BTCUSDT', 'ETHUSDT', 'USDTUSDT', 'BNBUSDT'])
Pulling data for SOLUSDT...
dict_keys(['BTCUSDT', 'ETHUSDT', 'USDTUSDT', 'BNBUSDT', 'SOLUSDT'])
Pulling data for USDCUSDT...
dict_keys(['BTCUSDT', 'ETHUSDT', 'USDTUSDT', 'BNBUSDT', 'SOLUSDT', 'USDCUSDT'])
Pulling data for XRPUSDT...
dict_keys(['BTCUSDT', 'ETHUSDT', 'USDTUSDT', 'BNBUSDT', 'SOLUSDT', 'USDCUSDT', 'XRPUSDT'])
Pulling data for DOGEUSDT...
dict_keys(['BTCUSDT', 'ETHUSDT', 'USDTUSDT', 'BNBUSDT', 'SOLUSDT', 'USDCUSDT', 'XRPUSDT', 'DOGEUSDT'])
Pulling data for TONUSDT...
dict_keys(['BTCUSDT', 'ETHUSDT', 'USDTUSDT', 'BNBUSDT', 'SOLUSDT', 'USDCUSDT', 'XRPUSDT', 'DOGEUSDT', 'TONUSDT'])
Pulling data for ADAUSDT...
dict_keys(['BTCUSDT', 'ETHUSDT', 'USDTUSDT', 'BNBUSDT', 'SOLUSDT', 'USDCUSDT', 'XRPUSDT', 'DOGEUSDT',

In [10]:
def drop_empty_dicts(data):
    return {k: v for k, v in data.items() if not v.empty}
def combine_dataframes(data):
    combined_df = pd.DataFrame()

    for symbol, df in data.items():
        if not df.empty:
            df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
            df.set_index('Open time', inplace=True)
            df = df[['Close']]  # Keep only the 'Close' column
            df.rename(columns={'Close': f'{symbol}_Close'}, inplace=True)  # Rename 'Close' to '{symbol}_Close'
            if combined_df.empty:
                combined_df = df
            else:
                combined_df = combined_df.join(df, how='outer')  # Join on index (Open time)

    return combined_df

cleaned_data = drop_empty_dicts(data)
merged = combine_dataframes(cleaned_data)


In [16]:
merged=merged.backfill()

In [153]:
momentum_7d = merged.pct_change(periods=7)
momentum_1d = merged.pct_change(periods=1)

# Calculate returns over the required period
returns_7d = merged.pct_change(periods=7)
returns_1d = merged.pct_change(periods=1)

# Calculate moving averages for additional indicators
sma_20 = merged.rolling(window=20).mean()
sma_50 = merged.rolling(window=50).mean()

# Generate buy and sell signals with refined return conditions and additional indicators
buy_signal = (momentum_7d.rank(axis=1, ascending=True) <= 3)
sell_signal = (momentum_1d.rank(axis=1, ascending=True) <= 3)

# Initialize signals DataFrame with the same structure
buy_entries = pd.DataFrame(0, index=merged.index, columns=merged.columns)
buy_exits = pd.DataFrame(0, index=merged.index, columns=merged.columns)
sell_entries = pd.DataFrame(0, index=merged.index, columns=merged.columns)
sell_exits = pd.DataFrame(0, index=merged.index, columns=merged.columns)

# Set buy and sell signals
for col in merged.columns:
    buy_entries[col] = buy_signal[col].astype(int)
    buy_exits[col] = (~buy_signal[col]).astype(int)
    sell_entries[col] = sell_signal[col].astype(int)
    sell_exits[col] = (~sell_signal[col]).astype(int)

# Define position sizing based on volatility
volatility = merged.pct_change().rolling(window=30).std()
size = 0.01 / volatility

# VectorBT portfolio
pf = vbt.Portfolio.from_signals(
    close=merged,
    entries=buy_entries,
    # exits=buy_exits,
    short_entries=sell_entries,
    # short_exits=sell_exits,
    upon_dir_conflict='short',
    # tp_stop=0.1,
    sl_stop=0.01,
    sl_trail=True,
    fees=0.005,
    init_cash=10000,
    freq='1d',
    size=size
)

# Run the portfolio
stats = pf.stats()
print(stats)


Start                                 2023-01-29 00:00:00
End                                   2024-05-26 00:00:00
Period                                  484 days 00:00:00
Start Value                                       10000.0
End Value                                     9993.764006
Total Return [%]                                 -0.06236
Benchmark Return [%]                            88.940858
Max Gross Exposure [%]                           1.563036
Total Fees Paid                                  4.088996
Max Drawdown [%]                                 0.094261
Max Drawdown Duration         289 days 03:25:42.857142856
Total Trades                                    10.634286
Total Closed Trades                             10.588571
Total Open Trades                                0.045714
Open Trade PnL                                  -0.001344
Win Rate [%]                                    33.934087
Best Trade [%]                                  12.688694
Worst Trade [%

In [149]:
pf.value().vbt.plot()


FigureWidget({
    'data': [{'name': 'BTCUSDT_Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f7c218b6-5d0d-4604-8e57-baa7fcde80e7',
              'x': array([datetime.datetime(2023, 1, 29, 0, 0),
                          datetime.datetime(2023, 1, 30, 0, 0),
                          datetime.datetime(2023, 1, 31, 0, 0), ...,
                          datetime.datetime(2024, 5, 24, 0, 0),
                          datetime.datetime(2024, 5, 25, 0, 0),
                          datetime.datetime(2024, 5, 26, 0, 0)], dtype=object),
              'y': array([10000.        , 10000.        , 10000.        , ...,  9962.21390747,
                           9962.21390747,  9962.21390747])},
             {'name': 'ETHUSDT_Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '60c30897-8222-49ad-96c7-5fe6c6a7513c',
              'x': array([datetime.datetime(2023, 1, 29, 0, 0),
                          